In [14]:
import asyncio
from copra.websocket import Channel, Client
import matplotlib.pyplot as plt
from collections import OrderedDict
from time import sleep
from dateutil import parser
import copy
import datetime
import itertools
from operator import itemgetter
import numpy as np
from dateutil.tz import tzutc
import math
import pytz
from pytz import timezone
import pickle
import pylab
from scipy.stats import probplot,expon,kstest,norm
import matplotlib.pyplot as plt
import json
from datetime import timedelta
import pandas as pd
from scipy.stats.stats import pearsonr
import bisect
import scipy.stats as ss
import bisect

# Load Data

In [2]:
def parse_file_json(file_name):
    with open(file_name, "r") as f:
        data = [json.loads(line) for line in f]
    starting_time = parser.parse(data[0][0])
    starting_bids = data[0][1]
    starting_asks = data[0][2]
    
    raw_updates = data[1:]
    
    Bids = {}
    Asks = {}    
    
    updates = []
    
    for price, amount in starting_bids:
        Bids[int(round((float(price)*100)))] = float(amount)
    for price, amount in starting_asks:
        Asks[int(round((float(price)*100)))] = float(amount)
        
    for u in raw_updates:
        price = int(round((float(u["price"])*100)))  
        side = u["side"]
        amount = float(u["amount"])
        time = parser.parse(u["time"])
        
        if side == "buy":
            change = amount - Bids.get(price,0)
        else:
            change = amount - Asks.get(price,0)
        
        updates.append({\
            "Bids": copy.copy(Bids), \
            "Asks": copy.copy(Asks), \
            "time": time, \
            "side": side, \
            "price": price, \
            "change": change            
        })
        
        if side == "buy":
            if u["amount"] == "0":
                del Bids[price]
            else:
                Bids[price] = float(amount)
        else:
            if u["amount"] == "0":
                del Asks[price]
            else:
                Asks[price] = float(amount)   

    return(starting_time, updates)

In [3]:
def shortened_updates(file_name,K):
    starting_time, updates = parse_file_json(file_name)
    
    res = []
    
    # Caluclate first reference price
    starting_bids = updates[0]['Bids']
    starting_asks = updates[0]['Asks']
    sorted_bids = list(reversed(sorted(starting_bids.items())))
    sorted_asks = list(sorted(starting_asks.items()))        
    best_bid = sorted_bids[0][0]
    best_ask = sorted_asks[0][0]
    if ((best_bid + best_ask) % 2) != 0:
        old_reference_price = round((best_bid+best_ask)/2, 1)
    else:
        old_reference_price = round((best_bid+best_ask+1)/2, 1)
            
    for u in updates:      
        # Find reference price
        sorted_bids = list(reversed(sorted(u['Bids'].items())))
        sorted_asks = list(sorted(u['Asks'].items()))        
        best_bid = sorted_bids[0][0]
        best_ask = sorted_asks[0][0]
        if ((best_bid + best_ask) % 2) != 0:
            reference_price = round((best_bid+best_ask)/2, 1)
        else:
            middle = (best_bid+best_ask)/2
            if old_reference_price > middle:
                reference_price = round((best_bid+best_ask)/2 + 0.5,1)
            else:
                old_reference_price = round((best_bid+best_ask)/2 - 0.5,1)
                
        shortened_book = OrderedDict([(k,0) for k in range(-K,K+1) if k != 0])
        first_bid = int(round(reference_price - 0.5))
        first_ask = int(round(reference_price + 0.5))        
        for k in range(-K,0):
            shortened_book[k] = u['Bids'].get(first_bid + k + 1,0)
        for k in range(1,K+1):
            shortened_book[k] = u['Asks'].get(first_ask + k - 1,0)
            
        # Find k from the price. Keep track of event if
        # abs(k) <= K
        price = u["price"]
        k = price - reference_price
        if k < 0:
            k = int(round(k - 0.5))
        else:
            k = int(round(k + 0.5))
        if abs(k) <= K:                 
            res.append({
                'reference_price': reference_price,
                'LOB': copy.copy(shortened_book),
                'k': k,
                'change': u['change'],
                'time': u['time']
            })
        
        old_reference_price = reference_price

    return starting_time, res

# Combine Updates that Occur At Same Time

# Combine Orders that Occur in Quick Succession
### (Orders that occur within 0.01 seconds of each other)

In [4]:
def processed_updates(file_name,K):
    starting_time,updates = shortened_updates(file_name,10)
    grouped_by_time = OrderedDict([(k, list(v)) for k, v in itertools.groupby(updates, key=lambda x:x['time'])])
    # Contains dictionary of time, reference price, order book, list of changes
    cleaned_updates = []
    for t, us in grouped_by_time.items():
        if len(us) == 1:
            u = us[0]
            cleaned_updates.append(copy.copy(u))
        else:
            new_update = {'time': t}
            us = sorted(us, key=lambda u:-abs(u['k']))
            grouped_by_k = OrderedDict((k, list(v)) for k, v in itertools.groupby(us, key=lambda u:u['k']))
            reference_k = list(grouped_by_k.keys())[0]
            new_update['reference_price'] = grouped_by_k[reference_k][0]['reference_price']
            new_update['LOB'] = copy.copy(grouped_by_k[reference_k][0]['LOB'])
            events = []
            for k in grouped_by_k:
                combined_change = 0
                for u in grouped_by_k[k]:
                    combined_change = combined_change + u['change']
                events.append((k,combined_change))
            for k,change in events:
                new_update = copy.deepcopy(new_update)
                new_update['k'] = k
                new_update['change'] = change
                cleaned_updates.append(new_update)
    
    combined_updates = []
    i = 0
    while i < len(cleaned_updates):
        reference_price = cleaned_updates[i]['reference_price']
        j = i
        updates_at_reference = []
        while (j < len(cleaned_updates)) and (cleaned_updates[j]['reference_price'] == reference_price):
            updates_at_reference.append(cleaned_updates[j])
            j += 1
        updates_at_reference = sorted(updates_at_reference,key=lambda u:u['k'])
        grouped_by_k = OrderedDict((k, list(v)) for k, v in itertools.groupby(updates_at_reference, key=lambda u:u['k']))
        for k,us in grouped_by_k.items():
            us = copy.deepcopy(sorted(us,key=lambda u:u['time']))
            keep_index = [True for u in us]
            for m in reversed(range(1,len(us))):
                quick_same_order = (us[m]['time'] - us[m-1]['time']).total_seconds() < 0.01
                same_sign = (us[m]['change'] * us[m-1]['change']) > 0
                if quick_same_order and same_sign:
                    us[m-1]['change'] += us[m]['change']
                    keep_index[m] = False
            for (u,keep) in zip(us, keep_index):
                if keep:
                    combined_updates.append(u)
        i = j + 1

    combined_updates = sorted(combined_updates,key=lambda u:u['time'])
    
    ending_time = combined_updates[-1]['time']
    
    return starting_time, ending_time, combined_updates

starting_time, ending_time, combined_updates = processed_updates('12_28_18.json',10)
len(combined_updates)

305806

In [5]:
starting_times, ending_times, time_period_updates = zip(*[processed_updates(f,10) for f in ['12_28_18.json','12_29_18.json','12_30_18.json']])

In [64]:
with open('processed_data.pkl', 'wb') as f:
    pickle.dump([starting_times, ending_times, time_period_updates], f)

# START HERE

In [15]:
with open('processed_data.pkl', 'rb') as f:
    starting_times, ending_times, all_updates = pickle.load(f)

In [16]:
all_update_times = []
for updates in all_updates:
    all_update_times.append([u['time'] for u in updates])

# Sample intervals

In [17]:
def updates_during_period(updates, update_times, start, end):
    left_index = bisect.bisect_left(update_times, start)
    right_index = bisect.bisect_right(update_times, end)
    return updates[left_index: right_index + 1]

    
def updates_during_random_period(updates, update_times, starting_time, ending_time, size):
    total_length = (ending_time - starting_time).total_seconds()
    interval_start = starting_time + timedelta(seconds=np.random.uniform()*(total_length - size))
    return updates_during_period(updates, update_times, interval_start, interval_start + timedelta(seconds=size))

def updates_during_random_period_multiple_dates(size):
    probabilities = [(e-s).total_seconds() - size for (s,e) in zip(starting_times, ending_times)]
    probabilities = [p/sum(probabilities) for p in probabilities]
    I = np.random.choice(len(all_updates), p=probabilities)
    return updates_during_random_period(all_updates[I], all_update_times[I], starting_times[I], ending_times[I], size)

In [18]:
K = 10
positive_updates = OrderedDict([(k,[]) for k in range(-K,K+1) if k != 0])
negative_updates = OrderedDict([(k,[]) for k in range(-K,K+1) if k != 0])

for _ in range(160000):
    updates = updates_during_random_period_multiple_dates(60)
    num_positive = OrderedDict([(k,0) for k in range(-K,K+1) if k != 0])
    num_negative = OrderedDict([(k,0) for k in range(-K,K+1) if k != 0])
    
    for u in updates:
        if u['change'] > 0:
            num_positive[u['k']] += 1
        else:
            num_negative[u['k']] += 1
            
    for k in range(-K,K+1):
        if k != 0:
            positive_updates[k].append(num_positive[k])
            negative_updates[k].append(num_negative[k])

# Full Correlation Matrix

In [19]:
observations = []
for k in positive_updates:
    observations.append(positive_updates[k])
for k in negative_updates:
    observations.append(negative_updates[k])
sigma = np.corrcoef(observations)
sigma

array([[ 1.        ,  0.37311832,  0.24789168, ...,  0.08576925,
         0.08849645,  0.0975215 ],
       [ 0.37311832,  1.        ,  0.44831511, ...,  0.09484749,
         0.16578818,  0.10438081],
       [ 0.24789168,  0.44831511,  1.        , ...,  0.11314765,
         0.2231125 ,  0.07931452],
       ..., 
       [ 0.08576925,  0.09484749,  0.11314765, ...,  1.        ,
         0.2505426 ,  0.14092613],
       [ 0.08849645,  0.16578818,  0.2231125 , ...,  0.2505426 ,
         1.        ,  0.24729873],
       [ 0.0975215 ,  0.10438081,  0.07931452, ...,  0.14092613,
         0.24729873,  1.        ]])

In [20]:
df = pd.DataFrame(positive_updates)
pos_pos_correlations = df.corr().astype('object')
pos_pos_correlations_styled = pos_pos_correlations.loc[-10:10,-10:10].style.format(lambda x: "{0:.3f}".format(x))
pos_pos_correlations_styled

,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10
-10,1.000,0.373,0.248,0.217,0.184,0.149,0.126,0.138,0.051,0.069,0.091,0.062,0.040,0.043,-0.003,0.018,0.070,0.075,0.085,0.099
-9,0.373,1.000,0.448,0.301,0.229,0.169,0.131,0.144,0.110,0.125,0.152,0.128,0.083,0.045,0.017,0.024,0.067,0.089,0.156,0.120
-8,0.248,0.448,1.000,0.588,0.430,0.264,0.197,0.226,0.099,0.135,0.157,0.146,0.087,0.040,0.010,0.032,0.069,0.091,0.203,0.091
-7,0.217,0.301,0.588,1.000,0.627,0.313,0.154,0.188,0.043,0.140,0.140,0.155,0.123,0.022,0.016,-0.020,0.026,0.104,0.118,0.060
-6,0.184,0.229,0.430,0.627,1.000,0.632,0.294,0.227,0.042,0.127,0.188,0.200,0.175,0.094,-0.004,-0.015,0.036,0.103,0.128,0.070
-5,0.149,0.169,0.264,0.313,0.632,1.000,0.527,0.341,0.141,0.202,0.252,0.215,0.215,0.155,0.089,0.057,0.068,0.087,0.131,0.071
-4,0.126,0.131,0.197,0.154,0.294,0.527,1.000,0.539,0.230,0.227,0.234,0.227,0.225,0.203,0.150,0.146,0.140,0.109,0.159,0.101
-3,0.138,0.144,0.226,0.188,0.227,0.341,0.539,1.000,0.394,0.252,0.241,0.254,0.241,0.211,0.135,0.141,0.187,0.141,0.200,0.107
-2,0.051,0.110,0.099,0.043,0.042,0.141,0.230,0.394,1.000,0.335,0.219,0.201,0.165,0.135,0.131,0.145,0.177,0.120,0.169,0.098
-1,0.069,0.125,0.135,0.140,0.127,0.202,0.227,0.252,0.335,1.000,0.269,0.204,0.196,0.233,0.225,0.186,0.199,0.147,0.167,0.098


In [21]:
pos_neg_correlations = pos_pos_correlations.copy()
for i in range(-K,K+1):
    for j in range(-K,K+1):
        if i != 0 and j != 0:
            pos_neg_correlations[i][j] = np.corrcoef(positive_updates[i], negative_updates[j])[0][1]
pos_neg_correlations_styled = pos_neg_correlations.loc[-10:10,-10:10].style.format(lambda x: "{0:.3f}".format(x))
pos_neg_correlations_styled

,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10
-10,0.672,0.354,0.267,0.210,0.194,0.163,0.146,0.178,0.112,0.115,0.123,0.116,0.089,0.083,0.027,0.054,0.095,0.102,0.119,0.094
-9,0.348,0.823,0.440,0.279,0.217,0.172,0.150,0.150,0.119,0.143,0.125,0.141,0.112,0.079,0.039,0.044,0.078,0.100,0.174,0.115
-8,0.222,0.453,0.901,0.583,0.416,0.249,0.177,0.188,0.094,0.159,0.138,0.155,0.110,0.051,0.015,0.039,0.070,0.094,0.179,0.083
-7,0.208,0.285,0.590,0.960,0.620,0.310,0.148,0.176,0.051,0.159,0.126,0.155,0.128,0.031,0.018,-0.018,0.028,0.115,0.132,0.065
-6,0.177,0.222,0.426,0.620,0.964,0.635,0.288,0.210,0.038,0.134,0.169,0.199,0.181,0.095,0.002,-0.012,0.034,0.105,0.125,0.075
-5,0.147,0.174,0.269,0.314,0.636,0.971,0.539,0.350,0.154,0.223,0.244,0.228,0.233,0.171,0.108,0.078,0.089,0.103,0.148,0.083
-4,0.131,0.144,0.200,0.153,0.287,0.531,0.969,0.539,0.235,0.238,0.228,0.234,0.237,0.212,0.158,0.162,0.151,0.113,0.165,0.105
-3,0.143,0.156,0.234,0.193,0.236,0.345,0.553,0.968,0.422,0.260,0.247,0.268,0.253,0.221,0.137,0.143,0.197,0.155,0.212,0.115
-2,0.088,0.146,0.148,0.098,0.100,0.199,0.296,0.490,0.948,0.390,0.264,0.228,0.198,0.169,0.158,0.174,0.211,0.142,0.197,0.109
-1,0.087,0.125,0.161,0.148,0.166,0.220,0.232,0.260,0.313,0.902,0.293,0.159,0.139,0.194,0.187,0.141,0.139,0.103,0.133,0.094


In [22]:
df = pd.DataFrame(negative_updates)
neg_neg_correlations = df.corr().astype('object')
neg_neg_correlations_styled = neg_neg_correlations.loc[-10:10,-10:10].style.format(lambda x: "{0:.3f}".format(x))
neg_neg_correlations_styled

,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10
-10,1.000,0.369,0.254,0.216,0.186,0.168,0.153,0.190,0.140,0.108,0.100,0.134,0.099,0.080,0.023,0.055,0.111,0.107,0.127,0.087
-9,0.369,1.000,0.476,0.271,0.217,0.184,0.162,0.160,0.145,0.113,0.145,0.169,0.113,0.077,0.031,0.043,0.094,0.092,0.166,0.100
-8,0.254,0.476,1.000,0.588,0.412,0.259,0.179,0.198,0.133,0.151,0.131,0.183,0.112,0.053,0.009,0.038,0.079,0.098,0.182,0.069
-7,0.216,0.271,0.588,1.000,0.614,0.310,0.145,0.182,0.097,0.149,0.102,0.173,0.127,0.034,0.016,-0.014,0.044,0.133,0.149,0.056
-6,0.186,0.217,0.412,0.614,1.000,0.642,0.279,0.214,0.082,0.150,0.132,0.225,0.180,0.095,0.001,-0.010,0.054,0.123,0.155,0.075
-5,0.168,0.184,0.259,0.310,0.642,1.000,0.544,0.350,0.207,0.218,0.204,0.259,0.239,0.174,0.105,0.077,0.102,0.115,0.159,0.081
-4,0.153,0.162,0.179,0.145,0.279,0.544,1.000,0.547,0.292,0.223,0.196,0.267,0.247,0.220,0.156,0.160,0.155,0.122,0.172,0.103
-3,0.190,0.160,0.198,0.182,0.214,0.350,0.547,1.000,0.507,0.256,0.191,0.290,0.265,0.228,0.136,0.152,0.213,0.174,0.222,0.109
-2,0.140,0.145,0.133,0.097,0.082,0.207,0.292,0.507,1.000,0.374,0.229,0.240,0.215,0.173,0.154,0.183,0.224,0.154,0.184,0.103
-1,0.108,0.113,0.151,0.149,0.150,0.218,0.223,0.256,0.374,1.000,0.247,0.156,0.145,0.190,0.191,0.160,0.173,0.121,0.144,0.094


# Find Average Event Sizes and Average Rate

In [24]:
K = 10
def AES_and_Rate(updates, starting_times):
    event_sizes_pos = OrderedDict([(k,[]) for k in range(-K,K+1) if k != 0])
    event_sizes_neg = OrderedDict([(k,[]) for k in range(-K,K+1) if k != 0])
    
    for updates in all_updates:
        for u in updates:
            if u['change'] > 0:
                event_sizes_pos[u['k']].append(u['change'])
            else:
                event_sizes_neg[u['k']].append(-u['change'])
                
    total_time = np.sum([(e-s).total_seconds() for s,e in zip(starting_times,ending_times)])
                
    AESs = OrderedDict([((k,"pos"),0) for k in range(-K,K+1) if k != 0] + \
                       [((k,"neg"),0) for k in range(-K,K+1) if k != 0])
    numbers_of_events = OrderedDict([((k,"pos"),0) for k in range(-K,K+1) if k != 0] + \
                                   [((k,"neg"),0) for k in range(-K,K+1) if k != 0])
    average_rates = OrderedDict([((k,"pos"),0) for k in range(-K,K+1) if k != 0] + \
                                [((k,"neg"),0) for k in range(-K,K+1) if k != 0])
    
    for (k,sizes) in event_sizes_pos.items():
        AESs[(k,"pos")] = np.mean(sizes)
        numbers_of_events[(k,"pos")] = len(sizes)
        
    for (k,sizes) in event_sizes_neg.items():
        AESs[(k,"neg")] = np.mean(sizes)
        numbers_of_events[(k,"neg")] = len(sizes)
        
    for k in numbers_of_events:
        average_rates[k] = numbers_of_events[k] / total_time
        
    return AESs, numbers_of_events, average_rates
            

AESs, numbers_of_events, average_rates = AES_and_Rate(all_updates, starting_times)

for k in range(-K,K+1):
    if k != 0:
        print("k = {}".format(k))
        print("AES Positive: {}".format(AESs[(k,"pos")]))
        print("Average Rate: {}".format(average_rates[(k,"pos")]))
        print("Multiplied: {}".format(AESs[(k,"pos")]*average_rates[(k,"pos")]))
        print("AES Negative: {}".format(AESs[(k,"neg")]))
        print("Average Rate: {}".format(average_rates[(k,"neg")]))
        print("Multiplied: {}".format(AESs[(k,"neg")]*average_rates[(k,"neg")]))
        print()

k = -10
AES Positive: 601.4124541063055
Average Rate: 0.0015663206844764182
Multiplied: 0.9420047667684309
AES Negative: 597.758153757424
Average Rate: 0.0019019608311499365
Multiplied: 1.1369125949471217

k = -9
AES Positive: 479.6863613652646
Average Rate: 0.0026966949715493014
Multiplied: 1.29356779861449
AES Negative: 446.5999332116102
Average Rate: 0.002851012280364712
Multiplied: 1.273261893996361

k = -8
AES Positive: 584.5728342409202
Average Rate: 0.005995227447478704
Multiplied: 3.5046471008915834
AES Negative: 552.6849764006743
Average Rate: 0.006180408218057197
Multiplied: 3.4158187701434755

k = -7
AES Positive: 530.3406793258122
Average Rate: 0.021731735013930206
Multiplied: 11.525223110216285
AES Negative: 534.065776466186
Average Rate: 0.020898421546326987
Multiplied: 11.161131730056793

k = -6
AES Positive: 520.9486674717087
Average Rate: 0.050596787627852774
Multiplied: 26.358329093078943
AES Negative: 531.856717471623
Average Rate: 0.049582151322391445
Multiplied: 26

In [25]:
AESs_vector = []
average_rates_vector = []
for k in range(-K,K+1):
    if k != 0:
        AESs_vector.append(AESs[(k,"pos")])
        average_rates_vector.append(average_rates[(k,"pos")])
for k in range(-K,K+1):
    if k != 0:
        AESs_vector.append(AESs[(k,"neg")])
        average_rates_vector.append(average_rates[(k,"neg")])

In [41]:
def inverse_PDF_Poisson(average, u):
    x = 0
    p = np.exp(-average)
    s = p
    while u > s:
        x += 1
        p = p * average / x
        s = s + p
    return x  

def arrivals_during_time_period(AESs, average_rates, correlations, start_time, T):
    correlated_normals = np.random.multivariate_normal(mean=[0 for _ in AESs], cov=correlations)
    us = [norm.cdf(y) for y in correlated_normals]
    num_arrivals = [inverse_PDF_Poisson(rate*T,u) for (rate,u) in zip(average_rates,us)]
    labels = [(k,"pos") for k in range(-K,K+1) if k != 0] + [(k,"neg") for k in range(-K,K+1) if k != 0]
    events = []
    for i,n in enumerate(num_arrivals):
        position, direction = labels[i]
        for _ in range(n):
            arrival_time = start_time + np.random.uniform()*T
            arrival_size = np.random.exponential(AESs[i])
            if direction == "neg":
                arrival_size *= -1
            events.append((arrival_time, position, arrival_size))
            
    return sorted(events, key=lambda x: x[0])

In [42]:
events = arrivals_during_time_period(AESs_vector, average_rates_vector, sigma, 0, 60)
events

[(0.14517358187904783, 7, -280.1294486996599),
 (0.16160360673443064, -3, -212.7962687696645),
 (0.3420600454095202, 4, 469.9409762100495),
 (0.8095595195609939, 4, -102.43786025610089),
 (1.4832426595749304, -1, 220.72537992435858),
 (1.5105798425136774, -1, 111.33688690540629),
 (1.7437632771768241, 4, -236.17842775599604),
 (1.9757038179638187, 3, 185.5495186860809),
 (2.0131151277105364, -4, -2162.0100874212644),
 (2.4672437836763605, 6, -227.97727469182695),
 (2.7028709585922295, 2, -476.54810671624347),
 (2.7955681389550535, -1, -29.140520712167767),
 (3.187536455591451, 6, -251.42274649761254),
 (3.242025714991339, 4, 378.3942097656265),
 (3.2518803065301793, 4, 383.4850646360781),
 (3.418217921786153, 1, -23.716718283973822),
 (3.4481795333034437, 2, 253.4410480055676),
 (3.990115990454761, 3, -113.77374637370701),
 (4.130500380694984, 4, 2.763615904039501),
 (4.151632796323643, 6, -82.06615571531168),
 (5.448937310785846, -2, 59.463031660063045),
 (5.929250485176201, -3, 155.8